#📌 Extracão

In [8]:
import pandas as pd
import requests
import json


url_dados = 'https://raw.githubusercontent.com/jhonatanwsds/TelecomX_BR/refs/heads/main/TelecomX_Data.json'

try:
    response = requests.get(url_dados)
    response.raise_for_status()
    data = json.loads(response.text)

    dados_clientes = pd.json_normalize(data)

    print("Dados extraídos e organizados com sucesso da forma simples!")
    display(dados_clientes.head())

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados extraídos e organizados com sucesso da forma simples!


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformação

In [6]:
import pandas as pd
import requests

# --- ✅ Consultar o dicionário para entender as variáveis ---

url_dicionario = 'https://raw.githubusercontent.com/jhonatanwsds/TelecomX_BR/refs/heads/main/TelecomX_dicionario.md'

print("--- Dicionário de Dados (Consultado do GitHub) ---")
try:
    # Acessando  o link  do dicionário
    response = requests.get(url_dicionario)
    response.raise_for_status()
    print(response.text) # Imprime o dicionário na tela
except Exception as e:
    print(f"ERRO: Não foi possível carregar o dicionário do GitHub. Verifique o link. Erro: {e}")


# --- ✅ Explorar as colunas e verificar seus tipos de dados ---

print("\n\n--- Estrutura do Dataset (Colunas e Tipos de Dados) ---")

try:
    dados_clientes.info()
except NameError:
    print("ERRO: A tabela 'dados_clientes' não foi encontrada. Execute a célula de Extração primeiro.")

--- Dicionário de Dados (Consultado do GitHub) ---
#### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatu

Tratamento de dados:

In [12]:
import numpy as np

print("--- Corrigindo as Inconsistências ---")

# 1. Converte a cobrança total para número.

dados_clientes['account.Charges.Total'] = pd.to_numeric(dados_clientes['account.Charges.Total'], errors='coerce')

# 2. Troca o texto vazio ('') no Churn por um valor ausente (NaN).
dados_clientes['Churn'] = dados_clientes['Churn'].replace('', np.nan)

# 3. Remove todas as linhas que agora têm algum valor ausente.
dados_clientes.dropna(inplace=True)

print("\nDados corrigidos com sucesso!")

# 4. Verificação Final: O .info() prova que a limpeza funcionou.
print("\n--- Estrutura Final dos Dados ---")
dados_clientes.info()

--- Corrigindo as Inconsistências ---

Dados corrigidos com sucesso!

--- Estrutura Final dos Dados ---
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7032 non-null   object 
 1   Churn                      7032 non-null   object 
 2   customer.gender            7032 non-null   object 
 3   customer.SeniorCitizen     7032 non-null   int64  
 4   customer.Partner           7032 non-null   object 
 5   customer.Dependents        7032 non-null   object 
 6   customer.tenure            7032 non-null   int64  
 7   phone.PhoneService         7032 non-null   object 
 8   phone.MultipleLines        7032 non-null   object 
 9   internet.InternetService   7032 non-null   object 
 10  internet.OnlineSecurity    7032 non-null   object 
 11  internet.OnlineBackup      7032 non-null   object 
 12  inter

In [11]:
import pandas as pd

print("--- Verificando Inconsistências ---")

# 1. Checa se há valores ausentes
print("\n[1] Valores ausentes em cada coluna:")
print(dados_clientes.isnull().sum())

# 2. Checa se há clientes duplicados
duplicados = dados_clientes['customerID'].duplicated().sum()
print(f"\n[2] Quantidade de clientes duplicados: {duplicados}")

# 3. Mostra os valores únicos nas colunas de texto
print("\n[3] Valores únicos em cada categoria:")
colunas_texto = dados_clientes.select_dtypes(include=['object']).columns

for coluna in colunas_texto:
    if coluna != 'customerID':
        # Mostra os valores diferentes em cada coluna
        print(f"  - Coluna '{coluna}': {pd.unique(dados_clientes[coluna])}")

--- Verificando Inconsistências ---

[1] Valores ausentes em cada coluna:
customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
dtype: int64

[2] Quantidade de clientes duplicados: 0

[3] Valores únicos em cada categoria:
  - Coluna 'Churn': ['No' 'Yes' '']
  - Coluna 'customer.gender': ['Female' 'Male']
  - Coluna 'customer.Partner': ['Yes' 'No']
  - Coluna 'customer.Dependents': ['Yes' 'No']
  - Co

#📊 Carga e análise

#📄Relatorio Final